# Excursion

This is an excourse, not really meant to be read again, just me trying out things while reading the book. It is readable, but will include way more of my thoughts in the code thatn usual.

# Excourse on Linear Regression

## Normal Equation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt # type: ignore
from matplotlib.figure import Figure # type: ignore
from mpl_toolkits.mplot3d import Axes3D  # type: ignore
from sklearn.preprocessing import add_dummy_feature
from sklearn.metrics import mean_squared_error

# Taken from the function x1 + x2=y
X = np.array([[1,1],[2,2],[3,3]])
y = np.array([[2], [4], [6]])

# Intitialise rng 
rng  = np.random.default_rng(42)

# Will throw error because of perfectly correlated data, let us add some random noise beforehand
# If you add noise to your bias you get some really whacky stuff with gradient descent
X_b_noise = rng.random(size=(3,2))*0.1 
X_b = add_dummy_feature(X_b_noise)


print(X_b)


# So we used y = x1 + x2  to generate the data, let us see how close we get
theta_best = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

[[1.         0.0773956  0.04388784]
 [1.         0.08585979 0.0697368 ]
 [1.         0.00941773 0.09756224]]


In [23]:
# We would have hoped for one on both, but we basically get an equivalent feature by theta_2 being approx. 2
# Error is due to the nois
print(f"Theta/ Model Parameters{theta_best}")

# Make predictions for 2 isntances
X_samples_test = np.array([[4,4], [5,5]])
y_test = np.array([[8],[10]])

X_test = add_dummy_feature(X_samples_test)

y_pred = X_test @ theta_best

print(f"Predictions:{y_pred}")
error = mean_squared_error(y_test,y_pred)
print(error)


Theta/ Model Parameters[[-1.50836969]
 [ 1.7874769 ]
 [76.78724959]]
Predictions:[[312.79053628]
 [391.36526277]]
119168.36732462407


We can compute the feature weights (coefficients) and the bias (intercept) in the following way using sklearn and directly using the scipy function that the LinearRegression class is based on. Note that for the calculation of theta we use the pseudoinverse (Moore-Penrose) of X (see Hands on ML). This doesn't seem to be any less accurate, at least for our example, than the closed-form solution (Normal Equation).

The pseudoinverse is always defined, which makes it work on singular matirces as well. The calculation of the pseudoinverse uses Singular Value Decomposition (see Hands on ML)

In [24]:
# Doing it with sklearn
from sklearn.linear_model import LinearRegression

lin_mdl = LinearRegression()

lin_mdl.fit(X_b,y)
y_pred=lin_mdl.predict(X_test)
for i in range(y_pred.shape[0]):
	print(f"Prediction {i}: {y_pred[i,0]:.2f}")
print(y_pred)
# Equivalent terms
bias, weights =  lin_mdl.intercept_,lin_mdl.coef_



Prediction 0: 312.79
Prediction 1: 391.37
[[312.79053628]
 [391.36526277]]


### Implement matrix multiplication, small exercise   

In [25]:
def matrix_mult(A: np.ndarray,B: np.ndarray) -> np.ndarray:
    if not (A.shape[1] == B.shape[0]):
        raise ValueError("Incompatible Matrix Dimensions:{} {}".format(A.shape,B.shape) )

    C = np.zeros((A.shape[0],B.shape[1]))

    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            C[i,j] = np.sum(A[i,:]*B[:,j])
    
    return C

A = np.array([[1,1,1],[1,1,1],[1,1,1]])
A_test = np.array([[1,1,1],[1,1,1]])
B = np.array([[1,1,1],[1,1,1],[1,1,1]])
B_false = np.array([[1,1,1],[1,1,1]])
B_test = np.array([[1,1],[1,1],[1,1]])

print(matrix_mult(A,B))
try:
    print(matrix_mult(A,B_false))
except:
    print('It failed :(....As expected hahaha <evil> :)')

print(matrix_mult(A,B_test))
print(matrix_mult(A_test,B))




[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]
It failed :(....As expected hahaha <evil> :)
[[3. 3.]
 [3. 3.]
 [3. 3.]]
[[3. 3. 3.]
 [3. 3. 3.]]


# Implement gradient descent    

In [26]:

def gradient_descent(n_epochs:int, X: np.ndarray, y:np.ndarray, eta: float,theta: np.ndarray) -> np.ndarray:
    m = X.shape[0]
    i = 0
    while i<n_epochs:
        # gradient vetor, 
        grad_vec = 2/m * X.T @ (X@theta - y)
        theta = theta - eta*grad_vec
        i+=1
    
    return theta

for i in [X,X_b]:
    theta_rand = rng.random((i.shape[1],1)) *5

    eta=0.1
    n_epochs=1000
    # No need to add dummy parameter when using gradient descent apparently
    # Well a dummy parameter (bias) was added by geron, let me add it back
    weights = gradient_descent(n_epochs, i,y,eta,theta_rand)

    print(f"Input Matrix: \n {i}")
    print(f"Weights: \n {weights}")
    print(f"Predictions: \n {i @ weights}")

Input Matrix: 
 [[1 1]
 [2 2]
 [3 3]]
Weights: 
 [[0.93768849]
 [1.06231151]]
Predictions: 
 [[2.]
 [4.]
 [6.]]
Input Matrix: 
 [[1.         0.0773956  0.04388784]
 [1.         0.08585979 0.0697368 ]
 [1.         0.00941773 0.09756224]]
Weights: 
 [[ 3.72197331]
 [-5.55009057]
 [ 8.48822276]]
Predictions: 
 [[3.66495049]
 [3.8373852 ]
 [4.49783401]]
